In [1]:
from tqdm import tqdm

from eis1600.corpus_analysis.text_methods import get_text_as_list_of_mius
from eis1600.dates.month_methods import month_annotate_miu_text
from eis1600.processing.preprocessing import get_yml_and_miu_df
from eis1600.processing.postprocessing import merge_tagslists, reconstruct_miu_text_with_tags
from eis1600.yml.yml_handling import add_annotated_entities_to_yml

In [2]:
infile = '../../OpenITI_EIS1600_Texts/data/0902Sakhawi/0902Sakhawi.DawLamic/0902Sakhawi.DawLamic.ITO20230111-ara1.EIS1600'
mius_list = get_text_as_list_of_mius(infile)

In [3]:
res = []
error = ''
re_text = ''
for idx, tup in tqdm(list(enumerate(mius_list))):
    try:
        uid, miu_as_text, analyse_flag = tup
        yml_handler, df = get_yml_and_miu_df(miu_as_text)
        df['MONTH_TAGS'] = month_annotate_miu_text(df[['TOKENS']], uid)

        columns_of_automated_tags = ['MONTH_TAGS']
        for col in columns_of_automated_tags:
            if col in df.columns:
                df['TAGS_LISTS'] = df.apply(lambda x: merge_tagslists(x['TAGS_LISTS'], x[col]), axis=1)
        df_subset = df[['SECTIONS', 'TOKENS', 'TAGS_LISTS']]
        add_annotated_entities_to_yml(df_subset, yml_handler, uid)
        re_text += reconstruct_miu_text_with_tags(df_subset) + '\n\n'
    except Exception as e:
        uid, miu_as_text, analyse_flag = tup
        error += f'{uid}\n{e}\n\n\n'


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15686/15686 [15:09<00:00, 17.25it/s]


In [4]:
with open('../../0902Sakhawi.DawLamic.ITO20230111-ara1.EIS1600', 'w') as fh:
    fh.write(re_text)

In [2]:
from pandas import read_csv

In [5]:
df = read_csv('../eis1600/models/data/poetic_meters.csv')
df.loc[~df['POETRY'], 'METER']

2          الدوبيت
6          السلسلة
16        المواليا
19    شعر التفعيلة
20          شعر حر
21            عامي
Name: METER, dtype: object